In [1]:
%run TransformCoordinates.ipynb

class QualityControl(TransformCoordinates):
    
    def __init__(self, tray, position, points, trayPosition):
        TransformCoordinates.__init__(self, tray, position, points)
        self.trayPosition = trayPosition
        self.degrees = [89.5, 89.0, 88.0, 0.]
        self.classify = {'1': 'GOOD', '2': 'OK', '3': 'BAD', '4': 'VERY BAD'}
        self.centers = [0.025, 0.050, 0.100, 10.0]
        self.finalRating = ''
        self.angleNotes = ''
        self.centerNotes = ''
        
    def angleQuality(self):
        self.degQC = -1
        for i, d in enumerate(self.degrees):
            if abs(self.deg[0]) > d or abs(self.deg[1]) > d:
                self.angleNotes = "The angle of the sensor is " + self.classify[str(i)] + " in position " + self.trayPosition
                self.degQC = i
        
    def positionQuality(self):
        self.posQC = -1
        for i, p in enumerate(self.centers):
            if self.centerPosition < p:
                self.centerNotes = "The placement of the sensor is " + self.classify[str(i)] + " in position " + self.trayPosition
                self.posQC = i
                
    def Rating(self):
        self.angleQuality()
        self.positionQuality()
        rate = max(self.degQC, self.posQC)
        self.finalRating = self.classify[str(rate)]
        
        if 'BAD' in self.angleNotes:
            print(self.angleNotes)
        if 'BAD' in self.centerNotes:
            print(self.centerNotes)